In [ ]:
import numpy as np
from coverage import *
from datetime import datetime, timezone, timedelta

from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import branca
import folium

! mkdir tmp

In [ ]:
from landsat import Instrument, Platform, Scene

In [ ]:
ts = load.timescale()
now_utc = datetime.now(timezone.utc)
tom_utc = now_utc + timedelta(days=1, hours=0.5)

now_utc2 = datetime.fromisoformat(str(now_utc))

now_ts = ts.from_datetime(now_utc)
now_rt = ts.from_datetime(now_utc2)
assert now_ts == now_rt
tom_ts = ts.from_datetime(tom_utc)

In [ ]:
start_dt = datetime.fromisoformat(Scene.start_utc)

num_days = 1

tles = gen_sats(
    # sat_nos=[Platform.norad_id]
    sat_nos=[39084,49260]
)

inst = camera_model(
    name=Instrument.name, 
    fl=Instrument.focal_length_mm, 
    pitch=Instrument.pitch_um*1e-3, 
    h_pix=Instrument.rows, 
    v_pix=Instrument.cols, 
)

times = gen_times(
    start_yr=start_dt.year,
    start_mo=start_dt.month, 
    start_day=start_dt.day, 
    days=1, 
    step_min=Instrument.img_period)

xcell_size = ycell_size = .1


In [ ]:
## AOI selection
# could select by path/ row?
wrs2 = gpd.read_file('./WRS2_descending_0/WRS2_descending.shp')

## Filter by AOI - should pass to forecast function instead?
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# aoi =  world[world.name == "Brazil"].geometry
# aoi =  world[world.continent == "South America"].geometry
aoi = gpd.read_file('./aois/eastern_us.geojson').geometry
aoi

xmin, ymin, xmax, ymax= aoi.total_bounds
wrs2 = wrs2.cx[xmin: xmax, ymin: ymax]


In [ ]:
# wrs2.explore()

In [ ]:
# ssp_df.geometry.explore()

In [ ]:
## Batch FOV generation over N satellites
gdfs = []
for tle in tles:
    sat = tle[0]
    fov_df = forecast_fovs(sat, times, inst)
    gdfs.append(fov_df)

fov_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="epsg:4326")

## Filter shapes crossing anti-meridian - also in main function
## TODO: Switch to stactools solution
fov_df["lonspan"] = fov_df.bounds['maxx'] - fov_df.bounds['minx']

## Create cmap for unique satellites and create color column
sat_ids = list(fov_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)

fov_df['color'] = fov_df['id'].apply(cmap)

fov_df = fov_df[fov_df["lonspan"] < 20].copy()

for satname in fov_df.satellite.unique():
    fov_df[fov_df.satellite==satname].to_file("./tmp/{}_fovs.geojson".format(satname.replace(" ", "_")))
fov_df

In [ ]:
## Select AOI from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

## Filter by AOI - should pass to forecast function instead?
# aoi =  world[world.name == "Brazil"].geometry
# aoi =  world[world.continent == "South America"].geometry
aoi = world[world.name == "United States of America"]#.geometry

aoi = gpd.read_file('./aois/eastern_us.geojson').geometry
aoi

xmin, ymin, xmax, ymax= aoi.total_bounds
fov_df = fov_df.cx[xmin: xmax, ymin: ymax]

In [ ]:
## Coverage data analysis for single satellite/ batch of satellites

# 1) Create a grid of equally spaced points
grid, grid_shape = create_grid(aoi.total_bounds, xcell_size, ycell_size)

# 2) Add "n_visits" column to grid using sjoin/ dissolve
shapes = gpd.GeoDataFrame(fov_df.geometry)
merged = gpd.sjoin(shapes, grid, how='left', predicate="intersects")
merged['n_visits']=0 # this will be replaced with nan or positive int where n_visits > 0
dissolve = merged.dissolve(by="index_right", aggfunc="count") # no difference in count vs. sum here?
grid.loc[dissolve.index, 'n_visits'] = dissolve.n_visits.values

grid.to_file('./tmp/n_visits.geojson')
grid.n_visits.fillna(0).describe()

In [ ]:
## Plotting FOVs

## Make a folium map
m = fov_df.explore(color="color", tooltip=["satellite", "time"])

## Add WRS2
# folium.GeoJson(data=wrs2["geometry"], overlay=False).add_to(m)


## View or save
m.save("./tmp/fov_output.html")

In [ ]:
## Plotting Revisit Map

## Form 2D array of n_visits based on grid shape
img = np.rot90(grid.n_visits.values.reshape(grid_shape))

## Create colormap and apply to img
## TODO: Make this our geoTIFF item for STAC catalog
colormap = branca.colormap.step.viridis.scale(1, grid.n_visits.max())
# colormap = branca.colormap.step.viridis.scale(1, 2)

def colorfunc(x):
    if np.isnan(x):
        return (0,0,0,0)
    else:
        return colormap.rgba_bytes_tuple(x)

# Apply cmap to img array and rearrange for RGBA
cmap = np.vectorize(colorfunc)
rgba_img = np.array(cmap(img))
rgba_img = np.moveaxis(rgba_img, 0, 2)

# Update image corner bounds based on cell size
xmin, ymin, xmax, ymax= grid.total_bounds
xmin = xmin - xcell_size/2
ymin = ymin - ycell_size/2
xmax = xmax + xcell_size/2
ymax = ymax + ycell_size/2

m = folium.Map()
m.fit_bounds([[ymin, xmin], [ymax, xmax]])
m.add_child(folium.raster_layers.ImageOverlay(rgba_img, opacity=.4, mercator_project=True,# crs="EPSG:4326",
                                 bounds = [[ymin,xmin],[ymax,xmax]]))
colormap.add_to(m)
m.save("./tmp/coverage_output.html")